In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# 데이터 불러오기
file_path = "/content/drive/MyDrive/Colab Notebooks/DSL/EDA2_Education/data/유초중등학교개황(09-23).csv"  # 데이터 파일 경로를 설정하세요
df = pd.read_csv(file_path, encoding='euc-kr')

# 필요한 열만 선택
columns_to_keep = [
    '조사년도', '시도명', '학제유형명', '학교명', '개방ID',
    '유초중등학교개황_학생수', '유초중등학교개황_교원수',
    '유초중등학교개황_학급수', '유초중등학교개황_정규교실수'
]
df = df[columns_to_keep]
#초등학교 data만 추출(중고등은 이름이 다양한 것에 주의!)
df = df[df['학제유형명']=='초등학교']
# '정규교실수 - 학급수' 계산
df['정규교실수_차이'] = df['유초중등학교개황_정규교실수'] - df['유초중등학교개황_학급수']

# 교원수 피벗 테이블
pivot_teachers = pd.pivot_table(
    df,
    index=['시도명', '학제유형명', '개방ID', '학교명'],  # 행
    columns='조사년도',  # 열
    values='유초중등학교개황_교원수',  # 값
    aggfunc='sum'  # 값이 중복될 경우 합계
)
# 정규교실수 - 학급수 피벗 테이블
pivot_classroom_diff = pd.pivot_table(
    df,
    index=['시도명', '학제유형명', '개방ID', '학교명'],  # 행
    columns='조사년도',  # 열
    values='정규교실수_차이',  # 값
    aggfunc='sum'
)
# 학생수 피벗 테이블
pivot_students = pd.pivot_table(
    df,
    index=['시도명', '학제유형명', '개방ID', '학교명'],  # 행
    columns='조사년도',  # 열
    values='유초중등학교개황_학생수',  # 값
    aggfunc='sum'
)
pivot_students = pivot_students.fillna(0); pivot_classroom_diff = pivot_students.fillna(0); pivot_teachers = pivot_students.fillna(0)

<ipython-input-3-9b8d4977b096>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['정규교실수_차이'] = df['유초중등학교개황_정규교실수'] - df['유초중등학교개황_학급수']


In [ ]:
#pivot_students에 대해서

In [ ]:
#NaN이 정상값사이에 존재하는 학교 확인 -> 없음 -> 따라서 NaN은 끝쪽에만 있음, 학교가 개교전, 폐교후로 생각할 수 있음 -> 회귀분석시 NaN을 제외하고 분석하면 됨
def has_nan_between_valid_values(row):
    # 첫 번째와 마지막 유효값의 인덱스 찾기
    first_valid = row.first_valid_index()
    last_valid = row.last_valid_index()

    # 유효값 범위 내에 NaN이 있는지 확인
    if first_valid is not None and last_valid is not None:
        return row.loc[first_valid:last_valid].isna().any()
    return False
filtered_schools = pivot_students[pivot_students.apply(has_nan_between_valid_values, axis=1)]
print(filtered_schools)

Empty DataFrame
Columns: [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
Index: []


In [ ]:
#2023년에 0이나 NaN인 학교들 제거(폐교)
pivot_students = pivot_students[
    (pivot_students[2023] != 0) & (pivot_students[2023].notna())
]

In [ ]:
pivot_students.head()

조사년도                           2009    2010    2011    2012    2013    2014  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0     0.0     0.0   
          1059736549 화동초등학교    58.0    54.0    50.0    44.0    38.0    32.0   
          1110422253 정라초등학교  1269.0  1257.0  1213.0  1164.0  1078.0  1024.0   
          1112041771 성림초등학교  1292.0  1314.0  1311.0  1349.0  1299.0  1267.0   
          1128984749 속사초등학교    20.0    21.0    23.0    22.0    25.0    24.0   

조사년도                           2015    2016    2017    2018    2019    2020  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0   956.0  1568.0   
          1059736549 화동초등학교    34.0    43.0    44.0    44.0    41.0    35.0   
          1110422253 정라초등학교   959.0   898.0   840.0   759.0   709.0   678.0   
          1112041771 성림초등학교  1202.0  1151.0  1146.0  1115.0  1141.0  1152.0   
          1128984749 속사초등학교    21.0    27.0    34.0    35.0    49.0    45.0   

조사년도                           2021    2022    2023  
시도명 학제유형명 개방ID       학교명                             
강원  초등학교  1008023023 섬강초등학교  1242.0  1299.0  1360.0  
          1059736549 화동초등학교    31.0    24.0    18.0  
          1110422253 정라초등학교   630.0   567.0   512.0  
          1112041771 성림초등학교  1029.0   942.0   892.0  
          1128984749 속사초등학교    50.0    48.0    37.0

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [ ]:
# 각 학교별로 회귀분석을 수행하는 함수
def perform_regression(row):
    # 조사년도와 학생수 데이터를 가져옴
    valid_data = row.dropna()  # NaN 제거
    years = valid_data.index.astype(int)  # 조사년도
    students = valid_data.values  # 학생수

    # 데이터가 2개 이상일 때만 회귀분석 수행
    if len(years) > 1:
        # 선형 회귀분석
        slope, intercept, r_value, p_value, _ = linregress(years, students)

        # 2030년 예측값 계산
        prediction_2030 = slope * 2030 + intercept

        # 결과 반환
        return pd.Series({
            'R값': r_value,
            'p-value': p_value,
            '2030 예측값': prediction_2030
        })
    elif len(years)==1:
        #데이터가 1개인 경우
        single_year = years[0]  # 조사년도
        single_student = students[0]  # 학생수

        # 결과 반환: 현재 데이터를 그대로 사용
        return pd.Series({
            'R값': 1.0,
            'p-value': 1.0,
            '2030 예측값': single_student  # 단일 데이터를 그대로 사용
        })
    else:
        # 데이터가 부족하면 NaN 반환
        return pd.Series({'R값': np.nan, 'p-value': np.nan, '2030 예측값': np.nan})

# 데이터프레임에 조사년도 컬럼 추가
result = pivot_students.apply(perform_regression, axis=1)

# 결과를 기존 데이터프레임에 병합
pivot_students_with_results = pd.concat([pivot_students, result], axis=1)

In [ ]:
pivot_students_with_results.head()

2009    2010    2011    2012    2013    2014  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0     0.0     0.0   
          1059736549 화동초등학교    58.0    54.0    50.0    44.0    38.0    32.0   
          1110422253 정라초등학교  1269.0  1257.0  1213.0  1164.0  1078.0  1024.0   
          1112041771 성림초등학교  1292.0  1314.0  1311.0  1349.0  1299.0  1267.0   
          1128984749 속사초등학교    20.0    21.0    23.0    22.0    25.0    24.0   

                               2015    2016    2017    2018    2019    2020  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교     0.0     0.0     0.0     0.0   956.0  1568.0   
          1059736549 화동초등학교    34.0    43.0    44.0    44.0    41.0    35.0   
          1110422253 정라초등학교   959.0   898.0   840.0   759.0   709.0   678.0   
          1112041771 성림초등학교  1202.0  1151.0  1146.0  1115.0  1141.0  1152.0   
          1128984749 속사초등학교    21.0    27.0    34.0    35.0    49.0    45.0   

                               2021    2022    2023        R값       p-value  \
시도명 학제유형명 개방ID       학교명                                                      
강원  초등학교  1008023023 섬강초등학교  1242.0  1299.0  1360.0  0.817617  1.966956e-04   
          1059736549 화동초등학교    31.0    24.0    18.0 -0.813327  2.262707e-04   
          1110422253 정라초등학교   630.0   567.0   512.0 -0.997449  2.720477e-16   
          1112041771 성림초등학교  1029.0   942.0   892.0 -0.929669  5.320523e-07   
          1128984749 속사초등학교    50.0    48.0    37.0  0.870908  2.382372e-05   

                                2030 예측값  
시도명 학제유형명 개방ID       학교명                  
강원  초등학교  1008023023 섬강초등학교  2061.533333  
          1059736549 화동초등학교    11.933333  
          1110422253 정라초등학교   101.100000  
          1112041771 성림초등학교   770.266667  
          1128984749 속사초등학교    62.816667

In [ ]:
# 시도명과 학교명 조합으로 중복 여부 확인
duplicated_schools = (
    pivot_students_with_results.index.to_frame()
    .reset_index(drop=True)  # MultiIndex를 DataFrame으로 변환
    .duplicated(subset=['시도명', '학교명'], keep=False)  # 중복 확인
)

# 중복된 행 필터링
duplicated_rows = pivot_students_with_results[duplicated_schools.values]

# 결과 출력
if not duplicated_rows.empty:
    print("같은 시도명 안에서 학교명이 겹치는 행:")
    print(duplicated_rows)
else:
    print("같은 시도명 안에서 학교명이 겹치는 행이 없습니다.")

같은 시도명 안에서 학교명이 겹치는 행:
                               2009    2010    2011    2012   2013   2014  \
시도명 학제유형명 개방ID       학교명                                                    
강원  초등학교  1354878783 남산초등학교  1292.0  1182.0  1122.0  1038.0  938.0  928.0   
          1974684715 중앙초등학교   469.0   449.0   404.0   362.0  329.0  304.0   
          2237509548 북평초등학교   100.0   100.0   104.0    87.0   72.0   68.0   
          4330835282 남산초등학교   121.0   116.0   116.0   111.0   96.0   79.0   
          4392260551 교동초등학교   331.0   283.0   227.0   212.0  200.0  176.0   
...                             ...     ...     ...     ...    ...    ...   
전남  초등학교  7610483288 동강초등학교   163.0   144.0   113.0    99.0   91.0   83.0   
          8494698166 동강초등학교   114.0    97.0    95.0    72.0   64.0   57.0   
          9193285405 북일초등학교    42.0    42.0    53.0    53.0   54.0   51.0   
충남  초등학교  7981485366 금성초등학교    74.0    75.0    68.0    73.0   75.0   69.0   
          8127098739 금성초등학교   152.0   138.0   116.0  

In [ ]:
pivot_students_with_results.to_csv("pivot_students.csv", encoding='utf-8-sig')